### 서울시 교통사고 시각화

In [1]:
# Data 불러오기
import pandas as pd

traffic_seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [2]:
traffic_seoul.tail()

,년도,월,자치구명,발생건수,사망자수,부상자수
4495,2019,8,강동구,127,2,175
4496,2019,9,강동구,98,2,137
4497,2019,10,강동구,108,1,144
4498,2019,11,강동구,146,0,199
4499,2019,12,강동구,108,1,131


In [3]:
# 크기
traffic_seoul.shape

(4500, 6)

In [ ]:
# 내부 구조
traffic_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   년도      4500 non-null   int64 
 1   월       4500 non-null   int64 
 2   자치구명    4500 non-null   object
 3   발생건수    4500 non-null   int64 
 4   사망자수    4500 non-null   int64 
 5   부상자수    4500 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 211.1+ KB


In [6]:
# 기본 통계량 파악
traffic_seoul.describe()

,년도,월,발생건수,사망자수,부상자수
count,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000
mean,2012.000000,6.500000,134.198889,1.346444,191.292444
std,4.320974,3.452436,51.042497,1.263020,75.816072
min,2005.000000,1.000000,39.000000,0.000000,51.000000
25%,2008.000000,3.750000,101.000000,0.000000,141.000000
50%,2012.000000,6.500000,124.000000,1.000000,174.000000
75%,2016.000000,9.250000,152.000000,2.000000,218.000000
max,2019.000000,12.000000,383.000000,10.000000,552.000000


### 컬럼별 상관계수 구하기

In [ ]:
# 발생건수와 부상자수의 상관계수

import numpy as np

np.corrcoef(traffic_seoul.발생건수, traffic_seoul.부상자수)

array([[1.       , 0.9780286],
       [0.9780286, 1.       ]])

In [8]:
# 발생건수와 사망자수의 상관계수

np.corrcoef(traffic_seoul.발생건수, traffic_seoul.사망자수)

array([[1.        , 0.24622428],
       [0.24622428, 1.        ]])

In [9]:
# 부상자수와 사망자수의 상관계수

np.corrcoef(traffic_seoul.부상자수, traffic_seoul.사망자수)

array([[1.        , 0.24440625],
       [0.24440625, 1.        ]])

### 2019년 데이터만 추출

In [16]:
traffic_seoul_2019 = \
    traffic_seoul.loc[traffic_seoul.년도 == 2019, :]
traffic_seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158


In [17]:
# index 정리
traffic_seoul_2019.reset_index(drop=True, inplace=True)
traffic_seoul_2019.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158


### 자치구명별 발생건수, 사망자수, 부상자수 집계하기

In [22]:
traffic_anal = \
    pd.pivot_table(
        traffic_seoul,
        index='자치구명',
        aggfunc='sum'
    )
traffic_anal.head()

,년도,발생건수,부상자수,사망자수,월
자치구명,,,,,
강남구,362160,53250,77171,372,1170
강동구,362160,21529,29980,242,1170
강북구,362160,18708,25685,182,1170
강서구,362160,26883,38208,287,1170
관악구,362160,24797,34097,237,1170


In [23]:
# 필요없는 컬럼 제거하기 : 년도, 월

del traffic_anal['년도']
del traffic_anal['월']
traffic_anal.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,53250,77171,372
강동구,21529,29980,242
강북구,18708,25685,182
강서구,26883,38208,287
관악구,24797,34097,237


In [26]:
traffic_anal = \
    traffic_seoul.pivot_table(
        ['발생건수', '부상자수', '사망자수'],
        index='자치구명',
        aggfunc='sum'
    )
traffic_anal.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,53250,77171,372
강동구,21529,29980,242
강북구,18708,25685,182
강서구,26883,38208,287
관악구,24797,34097,237


### 지역별 지도 시각화
: 해당 지역의 경찰서 위치와 교통사고 발생건수의 크기를 원의 크기로 시각화 

In [29]:
# seoul.csv(위도 경도) 불러오기
seoul_limit = \
    pd.read_csv("../Data/seoul.csv", encoding='euc-kr') # 한국어 code: euc-kr, utf-8
seoul_limit.columns = ['자치구명', 'lon', 'lat']
seoul_limit.head()

,자치구명,lon,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


### traffic_anal과 seoul_limit 병합하기

In [31]:
data_result = \
    pd.merge(
        traffic_anal,
        seoul_limit,
        on='자치구명'
    )
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,53250,77171,372,127.0475,37.51731
1,강동구,21529,29980,242,127.1238,37.53013
2,강북구,18708,25685,182,127.0255,37.63975
3,강서구,26883,38208,287,126.8496,37.55094
4,관악구,24797,34097,237,126.9515,37.47834


In [32]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.3+ KB


### folium에서 계산하는 항목은 float type만 가능

In [33]:
data_result.발생건수 = data_result.발생건수.astype(float)
data_result.부상자수 = data_result.부상자수.astype(float)
data_result.사망자수 = data_result.사망자수.astype(float)

data_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lon     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.3+ KB


### 지도 시각화

In [34]:
import matplotlib.pyplot as plt
import folium

In [37]:
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lon,lat
0,강남구,53250.0,77171.0,372.0,127.0475,37.51731
1,강동구,21529.0,29980.0,242.0,127.1238,37.53013
2,강북구,18708.0,25685.0,182.0,127.0255,37.63975
3,강서구,26883.0,38208.0,287.0,126.8496,37.55094
4,관악구,24797.0,34097.0,237.0,126.9515,37.47834


In [47]:
# 서울 지도 만들기
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

# 자치구청의 위치를 지도로 표시
for i in data_result.index:
    # 자치구명 이름 정의
    officeName = folium.Popup(
        data_result.loc[i,'자치구명'],
        max_width=200
    )

    # Marker로 표시하기
    folium.Marker(
        [data_result.loc[i,'lat'], data_result.loc[i,'lon']],
        popup=officeName
    ).add_to(map)

    # Circle Marker로 데이터 크기(발생건수) 표시하기
    folium.CircleMarker(
        [data_result.loc[i,'lat'], data_result.loc[i,'lon']],
        radius=data_result.loc[i,'발생건수'] / 1000,
        color = 'blue',
        fill_color ='red'
    ).add_to(map)

map